# 神经网络结构，输入为3，输出为3

In [ ]:
import os

import torch
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.hidden1 = nn.Linear(3, 10)
        self.relu1 = nn.ReLU()
        self.hidden2 = nn.Linear(10, 10)
        self.relu2 = nn.ReLU()
        self.hidden3 = nn.Linear(10, 10)
        self.relu3 = nn.ReLU()
        self.output = nn.Linear(10, 3)

    def forward(self, x):
        x = self.hidden1(x)
        x = self.relu1(x)
        x = self.hidden2(x)
        x = self.relu2(x)
        x = self.hidden3(x)
        x = self.relu3(x)
        x = self.output(x)
        return x


In [ ]:
# 第一个出入口的类
class DataObj:
    def __init__(self, dic):
        self.__dict__.update(dic)
        self.time = dic["time"]
        self.x = dic["C123"]+dic["C12"]+dic["C1"]
        self.o = dic["C23"]+dic["C2"]
        self.d = dic["C1"]
        self.n = 0
        self.a = self.n + dic["C123"]+dic["C12"]+dic["C2"]+dic["C23"]
        self.y = dic["C123"]+dic["C12"]+dic["C2"]+dic["C23"]



# 第二个出入口的类
class DataObj2:
    def __init__(self, dic):
        self.__dict__.update(dic)
        self.time = dic["time"]
        self.x = dic["C123"]+dic["C12"]+dic["C2"]+dic["C23"]
        self.o = dic["C3"]
        self.d = dic["C12"]+dic["C2"]
        self.n = 0
        self.a = self.n + dic["C123"]+dic["C3"]+dic["C23"]
        self.y = dic["C123"]+dic["C23"]+dic["C3"]

# 每一个批次的a变化
def tn_a_recycle(datatn):
    # datatn中包含一个时间段内的不变数据，x、o、d、y、n不变，a随之变化,返回带a的结果
    temp = datatn
    for i in range(1,len(temp)):
        dtpre = temp[i-1]
        temp[i].a = dtpre.n+dtpre.x+dtpre.o-dtpre.d-dtpre.y
    return temp

# 最开始的n初始化计算，之后不变
# 按照天的时间进行计算，每天0点的n为0，后续时间的n通过计算得到，data[0]为0点数据


minN = 0
def generate_n(dataInit):
    dataInit[0].n = 0
    for i in range(1,len(dataInit)):
        dtpre = dataInit[i-1]
        dataInit[i].n = dtpre.n + dtpre.x+dtpre.o-dtpre.d-dtpre.y
#
def generate_n2(dataInit):
    # 遍历每一天找出出入都很少的
    for
    dataInit[0].n = 0
    for i in range(1,len(dataInit)):
        dtpre = dataInit[i-1]
        dataInit[i].n = dtpre.n + dtpre.x+dtpre.o-dtpre.d-dtpre.y

In [ ]:
import pandas as pd
import numpy as np
import random
import pickle

random.seed = 1234
init_n = [0,0]

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# 自定义数据集类
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)


with open("res/minute_res.pkl","rb") as f:
    df_res = pickle.load(f)
DO_list = [DataObj(x) for x in df_res]
DO_list2 = [DataObj2(x) for x in df_res]
# os.remove("res/modelpkl/")

In [ ]:
import torch.optim as optim
for recycle_i in range(20):
    print(f"第{recycle_i}次n生成——————————————————————————————————————")
    # 基于n的数据初始化
    DO_list[0].n = init_n[0]
    DO_list2[0].n = init_n[1]
    DOLL = [DO_list,DO_list2]
    for i in range(1,len(DO_list)-1):
        _ = DO_list[i-1]
        _2 = DO_list[i]
        _2.y = _.x+_.o+_.n-_.d
        _2.n = _2.x+_2.o+_.n-_2.d-_2.y

        _02 = DO_list2[i-1]
        _22 = DO_list2[i]
        _02.x = _.y
        _22.x = _2.y

        _22.y = _02.x+_02.o+_02.n-_02.d
        _22.n = _22.x+_22.o+_02.n-_22.d-_22.y


    datas = [[torch.tensor([data.x,data.o,data.a])/100,torch.tensor([data.y,data.d,data.n])/100] for data in DO_list]
    datas2= [[torch.tensor([data.x,data.o,data.a])/100,torch.tensor([data.y,data.d,data.n])/100] for data in DO_list2]
    data_collection = [datas,datas2]
    #model i
    for i in range(len(data_collection)):
        data = data_collection[i]
        # 创建数据集对象
        dataset = MyDataset(datas)
        # 定义划分比例
        train_ratio = 0.8  # 训练集比例
        test_ratio = 0.2  # 测试集比例

        # 计算划分的样本数量
        train_size = int(len(dataset) * train_ratio)
        test_size = len(dataset) - train_size

        # 使用random_split()函数划分数据集
        train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

        # 创建数据加载器
        train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
        pkl_path = f"res/modelpkl/model{i}.pkl"
        print(pkl_path)
        if os.path.exists(pkl_path):
            with open(pkl_path,"rb") as f:
                model = pickle.load(f)
        else:
            model = NeuralNetwork()

        # 创建优化器和损失函数
        learning_rate = 0.001
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)
        criterion = nn.CrossEntropyLoss()
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        # 迭代训练
        num_epochs = 10
        for epoch in range(num_epochs):
            for x, y in train_loader:
                # 将数据加载到设备（如GPU）上
                batch_data = x.to(device)
                batch_labels = y.to(device)
                # 前向传播
                outputs = model(batch_data)
                # 计算损失
                loss = criterion(outputs, batch_labels)

                # 反向传播和参数更新
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # 打印训练过程中的损失值
                print('Epoch [{}/{}], Batch Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        with open(pkl_path,"wb") as f:
            pickle.dump(model,f)
    for i in range(len(data_collection)-1):
        with open(f"res/modelpkl/model{i}.pkl","rb") as f:
            model = pickle.load(f)
        model.eval()
        # 进行预测
        with torch.no_grad():
            data1 = DOLL[i][0]
            init_n[i] = model((torch.tensor([data1.x,data1.o,data1.a])/100).to(device))[2]

# 预测，用n重新初始化